<a href="https://colab.research.google.com/github/paperspaceconsole1/Rotor-cuda/blob/main/classdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda")

print(device)

cuda


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]#because the images are 3 channel(mean,std)
)
trainset = torchvision.datasets.CIFAR10(root='/content/data',train=True,
                                        download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/content/data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:05<00:00, 30279468.35it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.cuda()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
PATH = 'cifar_net.pth'
try:
  net.load_state_dict(torch.load(PATH))
except:
  num_epochs=1
  for epoch in range(num_epochs):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs.to("cuda"))
        loss = criterion(outputs, labels.to("cuda"))
        loss.backward()
        optimizer.step()
    print(epoch,'/',num_epochs,loss)

print('Finished Training')

0 / 1 tensor(1.2891, device='cuda:0', grad_fn=<NllLossBackward0>)
Finished Training


In [7]:
torch.save(net.state_dict(), PATH)

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images=images.to("cuda")
        labels=labels.to("cuda")
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 47 %


In [25]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
#
# if it prints otherwise, then you need to enable GPU:
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [27]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Found existing installation: Pillow 9.4.0
Uninstalling Pillow-9.4.0:
  Successfully uninstalled Pillow-9.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-5.3.0-cp310-cp310-linux_x86_64.whl size=1014433 sha256=c5c54220d9d504ca5bd6bb3f997c22b5c72cb708d28724eb9405df275f5b4d85
  Stored in directory: /root/.cache/pip/wheels/e2/3a/40/5c297d58820b8ed94409be8d00c0cfc762e382b1d2284031f2
Successfully built Pillow
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 3.2.2 requires pillow>=7.1.0, but you have pillow 5.3.0 which is incompatible.
dopamine-rl 4.0.6 requires Pillow>=7.0.0, but you have pillow 5.3.0 which is incompatible.
fastai 2.7.12 requires pillow>6.0.0, but you have pillow 5.3.0 which is incompatible.
imageio 2.31.3 requires pillow>=8.3

AttributeError: ignored

In [28]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import time
import json
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL

from PIL import Image
from collections import OrderedDict

import os
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F


# defining the device to be run with
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [29]:
!pip install torch
!pip install torchvision

In [30]:
#!git clone https://github.com/manojkyresearch006/Image-Classification-Colab-and-PyTorch.git
import sys
sys.path.append("Image-Classification-Colab-and-PyTorch")

In [31]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import numpy as np

In [32]:

train_transform=transforms.Compose(
    [transforms.RandomCrop(32),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

valid_transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
valset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=valid_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=valid_transform)
valid_size=0.1
num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_indices=indices[split:]
valid_indices=indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)




trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler,num_workers=2)

valloader= torch.utils.data.DataLoader(valset, batch_size=batch_size, sampler=valid_sampler,num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [34]:
!python -m torch.utils.cpp_extension --cuda-extension --pytorch-extension --cxx-flags="-std=c++14" anaconda.py

In [35]:
!python3 -m torch.utils.cpp_extension --cuda-extension --pytorch-extension --cxx-flags="-std=c++14" anaconda.py

In [36]:
!python3 anaconda.py --cuda -t 2

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
searching... 15JNQnKpWu4s54x2GKKjwQp5HWiT3Em3RJ 0000000000000000000000000000000000000000000000151796ad9bb449e2f6 Speed: 138.57 keys/second
searching... 1DsK9CAn7k4ngT69ZuFuFejqhLiM826KdN 00000000000000000000000000000000000000000000001bfdfb5108d742118f Speed: 126.93 keys/second
searching... 1JYnr2a75x2d2kL7qFBPsa1sLG5yVfPXWm 00000000000000000000000000000000000000000000001a4fab9432f246f5b2 Speed: 126.93 keys/second
searching... 16WnscFUEvHndQWb3Lzd9FE7z23xKQmUnF 00000000000000000000000000000000000000000000001bdd06cc93b0797d64 Speed: 138.57 keys/second
searching... 12aFJhrrymTTVC8hesHa7C8g4MbW8wfkz7 00000000000000000000000000000000000000000000001892f545c9000a8188 Speed: 126.93 keys/second
searching... 1EMAEABGQAwqHWkrRQiyrRVX2tMm7mPhw2 00000000000000000000000000000000000000000000001549266e2ae2763d49 Speed: 138.57 keys/second
searching... 1PUHXhmLLKjKvuWbZpLApEspNsftiLMqmy 0000000000000000000000000000000000000000